<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-3-public/blob/main/Course%201%20-%20Custom%20Models%2C%20Layers%20and%20Loss%20Functions/Week%205%20-%20Callbacks/C1_W5_Lab_1_exploring-callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ungraded Lab: Introduction to Keras callbacks

In Keras, `Callback` is a Python class meant to be subclassed to provide specific functionality, with a set of methods called at various stages of training (including batch/epoch start and ends), testing, and predicting. Callbacks are useful to get a view on internal states and statistics of the model during training. The methods of the callbacks can  be called at different stages of training/evaluating/inference. Keras has available [callbacks](https://keras.io/api/callbacks/) and we'll show how you can use it in the following sections. Please click the **Open in Colab** badge above to complete this exercise in Colab. This will allow you to take advantage of the free GPU runtime (for faster training) and compatibility with all the packages needed in this notebook.

## Model methods that take callbacks
Users can supply a list of callbacks to the following `tf.keras.Model` methods:
* [`fit()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit), [`fit_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#fit_generator)
Trains the model for a fixed number of epochs (iterations over a dataset, or data yielded batch-by-batch by a Python generator).
* [`evaluate()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate), [`evaluate_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#evaluate_generator)
Evaluates the model for given data or data generator. Outputs the loss and metric values from the evaluation.
* [`predict()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict), [`predict_generator()`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model#predict_generator)
Generates output predictions for the input data or data generator.

## Imports

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

Version:  2.1.0


# Examples of Keras callback applications
The following section will guide you through creating simple [Callback](https://keras.io/api/callbacks/) applications.

In [2]:
# Download and prepare the horses or humans dataset

# horses_or_humans 3.0.0 has already been downloaded for you
path = "./tensorflow_datasets"
splits, info = tfds.load('horses_or_humans', data_dir=path, as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [6]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [7]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [8]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

## [TensorBoard](https://keras.io/api/callbacks/tensorboard/)

Enable visualizations for TensorBoard.

In [9]:
!rm -rf logs

In [10]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 21s 790ms/step - loss: 0.6795 - accuracy: 0.5426 - val_loss: 0.6600 - val_accuracy: 0.5659
Epoch 2/10
26/26 [==============================] - 20s 768ms/step - loss: 0.6219 - accuracy: 0.6484 - val_loss: 0.6277 - val_accuracy: 0.6049
Epoch 3/10
26/26 [==============================] - 19s 746ms/step - loss: 0.5637 - accuracy: 0.7226 - val_loss: 0.5877 - val_accuracy: 0.6341
Epoch 4/10
26/26 [==============================] - 19s 750ms/step - loss: 0.5467 - accuracy: 0.7324 - val_loss: 0.4935 - val_accuracy: 0.7707
Epoch 5/10
26/26 [==============================] - 19s 746ms/step - loss: 0.4807 - accuracy: 0.7725 - val_loss: 0.4467 - val_accuracy: 0.8049
Epoch 6/10
26/26 [==============================] - 20s 754ms/step - loss: 0.4484 - accuracy: 0.8139 - val_loss: 0.4136 - val_accuracy: 0.8293
Epoch 7/10
26/26 [==============================] - 20s 754ms/step - loss: 0.3761 - accuracy: 0.8540 - val_loss: 0.3599 - val_accuracy: 0.8878

In [11]:
%tensorboard --logdir logs

## [Model Checkpoint](https://keras.io/api/callbacks/model_checkpoint/)

Callback to save the Keras model or model weights at some frequency.

In [12]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5

Epoch 00001: saving model to weights.01-0.68.h5
26/26 - 20s - loss: 0.6736 - accuracy: 0.5827 - val_loss: 0.6813 - val_accuracy: 0.4878
Epoch 2/5

Epoch 00002: saving model to weights.02-0.71.h5
26/26 - 20s - loss: 0.6238 - accuracy: 0.6910 - val_loss: 0.7055 - val_accuracy: 0.4976
Epoch 3/5

Epoch 00003: saving model to weights.03-0.55.h5
26/26 - 20s - loss: 0.5797 - accuracy: 0.6837 - val_loss: 0.5542 - val_accuracy: 0.7512
Epoch 4/5

Epoch 00004: saving model to weights.04-0.47.h5
26/26 - 20s - loss: 0.5290 - accuracy: 0.7506 - val_loss: 0.4684 - val_accuracy: 0.8829
Epoch 5/5

Epoch 00005: saving model to weights.05-0.40.h5
26/26 - 20s - loss: 0.4918 - accuracy: 0.8212 - val_loss: 0.3964 - val_accuracy: 0.9073


In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])


Epoch 00001: saving model to saved_model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: saved_model/assets


INFO:tensorflow:Assets written to: saved_model/assets


26/26 - 21s - loss: 0.6774 - accuracy: 0.5633 - val_loss: 0.7006 - val_accuracy: 0.4537


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2

Epoch 00001: saving model to model.h5
26/26 - 20s - loss: 0.6797 - accuracy: 0.5876 - val_loss: 0.6779 - val_accuracy: 0.4537
Epoch 2/2

Epoch 00002: saving model to model.h5
26/26 - 20s - loss: 0.6079 - accuracy: 0.6934 - val_loss: 0.5804 - val_accuracy: 0.7951


## [Early stopping](https://keras.io/api/callbacks/early_stopping/)

Stop training when a monitored metric has stopped improving.

In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 20s - loss: 0.6772 - accuracy: 0.5669 - val_loss: 0.6655 - val_accuracy: 0.6732
Epoch 2/50
26/26 - 19s - loss: 0.6415 - accuracy: 0.6959 - val_loss: 0.6715 - val_accuracy: 0.4341
Epoch 3/50
26/26 - 20s - loss: 0.5965 - accuracy: 0.7117 - val_loss: 0.6743 - val_accuracy: 0.4585
Epoch 4/50
26/26 - 20s - loss: 0.5351 - accuracy: 0.7616 - val_loss: 0.4822 - val_accuracy: 0.8341
Epoch 5/50
26/26 - 20s - loss: 0.4762 - accuracy: 0.7871 - val_loss: 0.5195 - val_accuracy: 0.7415
Epoch 6/50
26/26 - 21s - loss: 0.4250 - accuracy: 0.8102 - val_loss: 0.3529 - val_accuracy: 0.8585
Epoch 7/50
26/26 - 20s - loss: 0.3637 - accuracy: 0.8528 - val_loss: 0.2842 - val_accuracy: 0.9073
Epoch 8/50
26/26 - 20s - loss: 0.3311 - accuracy: 0.8856 - val_loss: 0.2936 - val_accuracy: 0.8829
Epoch 9/50
26/26 - 21s - loss: 0.2608 - accuracy: 0.9088 - val_loss: 0.2221 - val_accuracy: 0.9073
Epoch 10/50
26/26 - 20s - loss: 0.2491 - accuracy: 0.9051 - val_loss: 0.1831 - val_accuracy: 0.9415
Epoch 11/

## [CSV Logger](https://keras.io/api/callbacks/csv_logger/)

Callback that streams epoch results to a CSV file.

In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 22s 859ms/step - loss: 0.6588 - accuracy: 0.6046 - val_loss: 0.6432 - val_accuracy: 0.6000
Epoch 2/5
26/26 [==============================] - 20s 763ms/step - loss: 0.6130 - accuracy: 0.7007 - val_loss: 0.6150 - val_accuracy: 0.6488
Epoch 3/5
26/26 [==============================] - 21s 788ms/step - loss: 0.5556 - accuracy: 0.7336 - val_loss: 0.5353 - val_accuracy: 0.7707
Epoch 4/5
26/26 [==============================] - 20s 773ms/step - loss: 0.5014 - accuracy: 0.7786 - val_loss: 0.5461 - val_accuracy: 0.7122
Epoch 5/5
26/26 [==============================] - 20s 750ms/step - loss: 0.4492 - accuracy: 0.8005 - val_loss: 0.4136 - val_accuracy: 0.8098


In [17]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.604623,0.658831,0.600000,0.643174
1,1,0.700730,0.613141,0.648780,0.614971
2,2,0.733577,0.555935,0.770732,0.535295
3,3,0.778589,0.500741,0.712195,0.546051
4,4,0.800487,0.449275,0.809756,0.413609


## [Learning Rate Scheduler](https://keras.io/api/callbacks/learning_rate_scheduler/)

Updates the learning rate during training.

In [18]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
Epoch 1/5
26/26 [==============================] - 37s 1s/step - loss: 0.6710 - accuracy: 0.6144 - val_loss: 0.6545 - val_accuracy: 0.5951

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
Epoch 2/5
26/26 [==============================] - 21s 813ms/step - loss: 0.6434 - accuracy: 0.6995 - val_loss: 0.6455 - val_accuracy: 0.6878

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
Epoch 3/5
26/26 [==============================] - 21s 821ms/step - loss: 0.6229 - accuracy: 0.7251 - val_loss: 0.6441 - val_accuracy: 0.6146

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
Epoch 4/5
26/26 [==============================] - 21s 793ms/step - loss: 0.6140 - accuracy: 0.6922 - val_loss: 0.6282 - val_accuracy: 0.7659

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
Epoch 5/5
26/26 [==============================] - 21s 819ms/step - loss: 0.6099

In [19]:
%tensorboard --logdir log_dir

## [ReduceLROnPlateau](https://keras.io/api/callbacks/reduce_lr_on_plateau/)

Reduce learning rate when a metric has stopped improving.

In [20]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 22s 836ms/step - loss: 0.6824 - accuracy: 0.5487 - val_loss: 0.6733 - val_accuracy: 0.5756
Epoch 2/50
26/26 [==============================] - 20s 772ms/step - loss: 0.6462 - accuracy: 0.6642 - val_loss: 0.6338 - val_accuracy: 0.6390
Epoch 3/50
26/26 [==============================] - 20s 773ms/step - loss: 0.6180 - accuracy: 0.7080 - val_loss: 0.5813 - val_accuracy: 0.7561
Epoch 4/50
26/26 [==============================] - 20s 774ms/step - loss: 0.5573 - accuracy: 0.7299 - val_loss: 0.5193 - val_accuracy: 0.7707
Epoch 5/50
26/26 [==============================] - 20s 777ms/step - loss: 0.5283 - accuracy: 0.7494 - val_loss: 0.5093 - val_accuracy: 0.7415
Epoch 6/50
26/26 [==============================] - 21s 796ms/step - loss: 0.4689 - accuracy: 0.7859 - val_loss: 0.4133 - val_accuracy: 0.8195
Epoch 7/50
26/26 [==============================] - 21s 827ms/step - loss: 0.4396 - accuracy: 0.7859 - val_loss: 0.3863 - val_accuracy: 0.8878

In [21]:
%tensorboard --logdir log_dir

Reusing TensorBoard on port 6007 (pid 4750), started 0:16:59 ago. (Use '!kill 4750' to kill it.)